<a href="https://colab.research.google.com/github/2019mohamed/Enzyme-classification/blob/master/CRaWl_(hyperparameter_in_progress).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install deepsnap

     |████████████████████████████████| 81kB 3.3MB/s 


In [ ]:
#! pip install torch-scatter 
! pip install torch-sparse 
#! pip install torch-cluster 
#! pip install torch-spline-conv 
#! pip install torch-geometric

  Created wheel for torch-sparse: filename=torch_sparse-0.6.9-cp37-cp37m-linux_x86_64.whl size=432311 sha256=0c2448cea1c13a1f57de39d23b713df1ad2e16c07d74cbcfbfe8cd91b2f0f29c
  Stored in directory: /root/.cache/pip/wheels/6c/9f/3e/8813c1f7e87d12b01b830d3506e7f3f6b044d9d53769057ddb
Successfully built torch-sparse


In [ ]:
import torch
import numpy as np
import torch
from torch_scatter import scatter_mean, scatter_sum

MAXINT = np.iinfo(np.int64).max
#https://github.com/toenshoff/CRaWl
import torch
import numpy as np
import torch_geometric as pygeo
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch_geometric.data import Data
from torch_scatter import scatter_sum


def preproc(data):
    """ Preprocess Pytorch Geometric data objects to be used with our walk generator """

    if data.num_edges == 0:
        return data

    if not data.is_coalesced():
        data.coalesce()

    if data.num_node_features == 0:
        data.x = torch.zeros((data.num_nodes, 1), dtype=torch.float32)

    if data.num_edge_features == 0:
        data.edge_attr = torch.zeros((data.num_edges, 1), dtype=torch.float32)

    edge_idx = data.edge_index
    edge_feat = data.edge_attr
    node_feat = data.x

    # remove isolated nodes
    if data.contains_isolated_nodes():
        edge_idx, edge_feat, mask = pygeo.utils.remove_isolated_nodes(edge_idx, edge_feat, data.num_nodes)
        node_feat = node_feat[mask]

    # Enforce undirected graphs
    if not pygeo.utils.is_undirected(edge_idx):
        x = edge_feat.detach().numpy()
        e = edge_idx.detach().numpy()
        x_map = {(e[0,i], e[1,i]): x[i] for i in range(e.shape[1])}
        edge_idx = pygeo.utils.to_undirected(edge_idx)
        e = edge_idx.detach().numpy()
        x = [x_map[(e[0,i], e[1,i])] if (e[0,i], e[1,i]) in x_map.keys() else x_map[(e[1,i], e[0,i])] for i in range(e.shape[1])]
        edge_feat = torch.tensor(x)

    data.edge_index = edge_idx
    data.edge_attr = edge_feat
    data.x = node_feat

    if not torch.is_tensor(data.y):
        data.y = torch.tensor(data.y)
    data.y = data.y.view(1, -1)

    order = node_feat.shape[0]

    # create bitwise encoding of adjacency matrix using 64-bit integers
    data.node_id = torch.arange(0, order)
    bit_id = torch.zeros((order, order // 63 + 1), dtype=torch.int64)
    bit_id[data.node_id, data.node_id // 63] = torch.tensor(1) << data.node_id % 63
    data.adj_bits = scatter_sum(bit_id[edge_idx[0]], edge_idx[1], dim=0)

    # compute node offsets in the adjacency list
    data.degrees = pygeo.utils.degree(edge_idx[0], dtype=torch.int64)
    adj_offset = torch.zeros((order,), dtype=torch.int64)
    adj_offset[1:] = torch.cumsum(data.degrees, dim=0)[:-1]
    data.adj_offset = adj_offset

    return data


def merge_batch(graph_data):
    """ Custom function to collate preprocessed data objects in the data loader """

    adj_offset = [d.adj_offset for d in graph_data]
    degrees = [d.degrees for d in graph_data]
    edge_idx = [d.edge_index for d in graph_data]

    num_nodes = torch.tensor([d.shape[0] for d in degrees])
    num_edges = torch.tensor([e.shape[1] for e in edge_idx])

    x_node = torch.cat([d.x for d in graph_data], dim=0)
    x_edge = torch.cat([d.edge_attr for d in graph_data], dim=0)
    x_edge = x_edge.view(x_edge.shape[0], -1)

    adj_offset = torch.cat(adj_offset)
    degrees = torch.cat(degrees)
    edge_idx = torch.cat(edge_idx, dim=1)

    node_graph_idx = torch.cat([i * torch.ones(x, dtype=torch.int64) for i, x in enumerate(num_nodes)])
    edge_graph_idx = torch.cat([i * torch.ones(x, dtype=torch.int64) for i, x in enumerate(num_edges)])

    node_shift = torch.zeros((len(graph_data),), dtype=torch.int64)
    edge_shift = torch.zeros((len(graph_data),), dtype=torch.int64)
    node_shift[1:] = torch.cumsum(num_nodes, dim=0)[:-1]
    edge_shift[1:] = torch.cumsum(num_edges, dim=0)[:-1]

    adj_offset += edge_shift[node_graph_idx]
    edge_idx += node_shift[edge_graph_idx].view(1, -1)

    graph_offset = node_shift

    adj_bits = [d.adj_bits for d in graph_data]
    max_enc_length = np.max([p.shape[1] for p in adj_bits])
    adj_bits = torch.cat([F.pad(b, (0,max_enc_length-b.shape[1],0,0), 'constant', 0) for b in adj_bits], dim=0)

    node_id = torch.cat([d.node_id for d in graph_data], dim=0)

    y = torch.cat([d.y for d in graph_data], dim=0)

    data = Data(x=x_node, edge_index=edge_idx, edge_attr=x_edge, y=y)
    data.batch = node_graph_idx
    data.adj_offset = adj_offset
    data.degrees = degrees
    data.graph_offset = graph_offset
    data.order = num_nodes
    data.node_id = node_id
    data.adj_bits = adj_bits
    return data


class CRaWlLoader(DataLoader):
    """ Custom Loader for our data objects """

    def __init__(self, dataset, batch_size=1, shuffle=False, **kwargs):
        super(CRaWlLoader, self).__init__(dataset, batch_size, shuffle, collate_fn=merge_batch, **kwargs)

class Walker(torch.nn.Module):

    def __init__(self, config):
        super(Walker, self).__init__()

        self.steps = config['steps']
        #self.train_start_ratio = config['train_start_ratio']
        self.win_size = config['win_size']

        self.compute_id = 'compute_id_feat' not in config.keys() or config['compute_id_feat']
        self.compute_adj = 'compute_adj_feat' not in config.keys() or config['compute_adj_feat']

        self.struc_feat_dim = 0
        if self.compute_id:
            self.struc_feat_dim += self.win_size
        if self.compute_adj:
            self.struc_feat_dim += self.win_size - 1

        self.choice_fct = Walker.uniform_choice if config['walk_mode'] == 'uniform' else Walker.nb_choice

    @staticmethod
    def sample_start(start_p, graph_idx, graph_offset, order, device):
        """
        Randomly sample start nodes
        :param start_p: Probability of starting a walk at a node
        :param graph_idx: Assignment of nodes to graphs
        :param graph_offset: Node list offset of each graph in the batch
        :param order: Nuber of nodes in each graph of the batch
        :param device: device to construct tensors on
        :return: A tensor of start vertices (index list) and an assignment to the graphs in the batch
        """

        num_graphs = order.shape[0]
        num_nodes = graph_idx.shape[0]
        num_walks = int(np.ceil(start_p * num_nodes))
        num_extra = num_walks - num_graphs

        idx = graph_offset + (torch.randint(0, MAXINT, (num_graphs,), device=device) % order)
        idx = torch.cat([idx, torch.randperm(num_nodes, device=device)[:num_extra]])

        choices = torch.randint(0, MAXINT, (num_walks,), device=device)
        start_graph = graph_idx[idx]
        start = graph_offset[start_graph] + (choices % order[start_graph])

        del idx, choices
        return start, start_graph

    @staticmethod
    def uniform_choice(i, walks, adj_nodes, adj_offset, choices, degrees, nb_degrees):
        """
        :param i: Index of the current step
        :param walks: Tensor of vertices in the walk
        :param adj_nodes: Adjacency List
        :param adj_offset: Node offset in the adjacency list
        :param choices: Cache of random integers
        :param degrees: Degree of each node
        :param nb_degrees: Reduced degrees for no-backtrack walks (not used here)
        :return: A list of a chosen outgoing edge for each walk
        """
        return adj_offset[walks[i]] + (choices[i] % degrees[walks[i]])

    @staticmethod
    def nb_choice(i, walks, adj_nodes, adj_offset, choices, degrees, nb_degrees):
        """
        :param i: Index of the current step
        :param walks: Tensor of vertices in the walk
        :param adj_nodes: Adjacency List
        :param adj_offset: Node offset in the adjacency list
        :param choices: Cache of random integers
        :param degrees: Degree of each node
        :param nb_degrees: Reduced degrees for no-backtrack walks
        :return: A list of a chosen outgoing edge for each walk
        """

        # do uniform step
        old_nodes = walks[i-1]
        cur_nodes = walks[i]
        edge_idx = choices[i] % degrees[cur_nodes]
        chosen_edges = adj_offset[cur_nodes] + edge_idx
        new_nodes = adj_nodes[chosen_edges]

        # correct backtracking
        bt = new_nodes == old_nodes
        if bt.max():
            bt_nodes = walks[i][bt]
            chosen_edges[bt] = adj_offset[bt_nodes] + (edge_idx[bt] + 1 + (choices[i][bt] % nb_degrees[bt_nodes])) % degrees[bt_nodes]
        return chosen_edges

    def sample_walks(self, data, x_edge, steps=None, start_p=1.0):
        """
        :param data: Preprocessed PyTorch Geometric data object.
        :param x_edge: Edge features
        :param steps: Number of walk steps (if None, default from config is used)
        :param start_p: Probability of starting a walk at each node
        :return: The data object with the walk added as an attribute
        """

        device = data.x.device

        # get adjacency data
        adj_nodes = data.edge_index[1]
        adj_offset = data.adj_offset
        degrees = data.degrees
        node_id = data.node_id
        adj_bits = data.adj_bits
        graph_idx = data.batch
        graph_offset = data.graph_offset
        order = data.order

        # use default number of steps if not specified
        if steps is None:
            steps = self.steps

        # set dimensions
        s = self.win_size
        n = degrees.shape[0]
        l = steps + 1

        # sample starting nodes
        if start_p < 1.0:
            start, walk_graph_idx = Walker.sample_start(start_p, graph_idx, graph_offset, order, device)
        else:
            start = torch.arange(0, n, dtype=torch.int64).view(-1)

        # init tensor to hold walk indices
        w = start.shape[0]
        walks = torch.zeros((l, w), dtype=torch.int64, device=device)
        walks[0] = start

        # get all random decisions at once (faster then individual calls)
        choices = torch.randint(0, MAXINT, (steps, w), device=device)

        # init feature tensors
        edge_feat = torch.zeros((l, w, x_edge.shape[1]), dtype=torch.float32, device=device)

        if self.compute_id:
            id_enc = torch.zeros((l, s, w), dtype=torch.bool, device=device)

        if self.compute_adj:
            edges = torch.zeros((l, s, w), dtype=torch.bool, device=device)

        # remove one choice of each node with deg > 1 for no_backtrack walks
        nb_degree_mask = (degrees == 1)
        nb_degrees = nb_degree_mask * degrees + (~nb_degree_mask) * (degrees - 1)

        for i in range(steps):
            # get next edges
            if i == 0:
                chosen_edges = self.uniform_choice(i, walks, adj_nodes, adj_offset, choices, degrees, nb_degrees)
            else:
                chosen_edges = self.choice_fct(i, walks, adj_nodes, adj_offset, choices, degrees, nb_degrees)

            # update nodes
            walks[i+1] = adj_nodes[chosen_edges]

            # update edge features
            edge_feat[i + 1] = x_edge[chosen_edges]

            o = min(s, i+1)
            prev = walks[i+1-o:i+1]

            if self.compute_id:
                # get local identity relation
                id_enc[i+1, s-o:] = torch.eq(walks[i+1].view(1, w), prev)

            if self.compute_adj:
                # look up edges in the bit-wise adjacency encoding
                cur_id = node_id[walks[i+1]]
                cur_int = (cur_id // 63).view(1, -1, 1).repeat(o, 1, 1)
                edges[i + 1, s - o:] = (torch.gather(adj_bits[prev], 2, cur_int).view(o,-1) >> (cur_id % 63).view(1,-1)) % 2 == 1

        # permute walks into the correct shapes
        data.walk_nodes = walks.permute(1, 0)

        # combine id, adj and edge features
        feat = [edge_feat.permute(1, 2, 0)]
        if self.compute_id:
            feat.append(torch._cast_Float(id_enc.permute(2, 1, 0)))
        if self.compute_adj:
            feat.append(torch._cast_Float(edges.permute(2, 1, 0))[:, :-1, :])
        data.walk_x = torch.cat(feat, dim=1)

        return data

class VNUpdate(torch.nn.Module):
    def __init__(self, dim, config):
        """
        Intermediate update layer for the virtual node
        :param dim: Dimension of the latent node embeddings
        :param config: Python Dict with the configuration of the CRaWl network
        """
        super(VNUpdate, self).__init__()

        self.mlp = torch.nn.Sequential(torch.nn.Linear(dim, dim, bias=False),
                                       torch.nn.BatchNorm1d(dim),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(config['dropout']),
                                       torch.nn.Linear(dim, dim, bias=False))

    def forward(self, data):
        x = scatter_sum(data.h, data.batch, dim=0)
        if data.vn_h is not None:
            x = x + data.vn_h
        data.vn_h = self.mlp(x)
        data.h = data.h + data.vn_h[data.batch]
        return data


class ConvModule(torch.nn.Module):
    def __init__(self, conv_dim, w_feat_dim, dim_in, dim_out, kernel_size, config):
        """
        :param conv_dim: Hidden dimension of the convolutions
        :param w_feat_dim: feature dimension of the walk feature tensors (without the node features)
        :param dim_in: Dimension of the latent node embedding used as input
        :param dim_out: Dimension of updated latent node embedding
        :param kernel_size: Kernel size of the convolutions
        :param config: Python Dict with the configuration of the CRaWl network
        """
        super(ConvModule, self).__init__()

        self.rescale = not dim_out == dim_in
        if self.rescale:
            self.rescale_op = torch.nn.Linear(dim_in, dim_out, bias=False)

        self.convs = torch.nn.Sequential(
            torch.nn.Conv1d(dim_in + w_feat_dim, conv_dim, kernel_size, padding=0, bias=False),
            torch.nn.BatchNorm1d(conv_dim),
            torch.nn.ReLU(),
            torch.nn.Conv1d(conv_dim, conv_dim, kernel_size, padding=0, bias=False),
            torch.nn.BatchNorm1d(conv_dim),
            torch.nn.ReLU())

        self.out = torch.nn.Sequential(torch.nn.Linear(conv_dim, 2 * dim_out, bias=False),
                                       torch.nn.BatchNorm1d(2 * dim_out),
                                       torch.nn.ReLU(),
                                       torch.nn.Dropout(config['dropout']),
                                       torch.nn.Linear(2 * dim_out, dim_out, bias=False))

    def forward(self, data):
        # rescale for the residual connection
        if self.rescale:
            h_r = self.rescale_op(data.h)
        else:
            h_r = data.h

        # build walk feature tensor
        h_c = torch.cat([data.h[data.walk_nodes].transpose(2, 1), data.walk_x], dim=1)

        # apply the cnn
        h_c = self.convs(h_c)

        # pool in walklet embeddings into nodes
        wl = h_c.shape[0] * h_c.shape[2]
        h = scatter_mean(h_c.transpose(2, 1).reshape(wl, -1), data.walk_nodes_flatt, dim=0, dim_size=data.num_nodes)

        # update embeddings
        h = self.out(h)
        data.h = h + h_r
        return data


class CRaWl(torch.nn.Module):
    def __init__(self, config ,node_feat_dim, edge_feat_dim, out_dim, node_feat_enc=None, edge_feat_enc=None):
        """
        :param model_dir: Directory to store model in
        :param config: Python Dict that specifies the configuration of the model
        :param node_feat_dim: Dimension of the node features
        :param edge_feat_dim: Dimension of the edge features
        :param out_dim: Output dimension
        :param node_feat_enc: Optional initial embedding of node features
        :param edge_feat_enc: Optional initial embedding of edge features
        """
        super(CRaWl, self).__init__()
        #self.model_dir = model_dir
        self.config = config
        self.out_dim = out_dim
        self.node_feat_enc = node_feat_enc
        self.edge_feat_enc = edge_feat_enc
        self.layers = config['layers']
        self.hidden = config['hidden_dim']
        self.kernel_size = config['kernel_size']
        self.dropout = config['dropout']
        self.residual = config['residual'] if 'residual' in config.keys() else True
        self.pool_op = config['pool'] if 'pool' in config.keys() else 'mean'
        self.vn = config['vn'] if 'vn' in config.keys() else False

        self.border = 2 * ((self.kernel_size - 1) // 2)
        self.tail = (self.kernel_size - 1) * 2

        self.walker = Walker(config)

        self.walk_dim = self.walker.struc_feat_dim + edge_feat_dim
        self.conv_dim = config['conv_dim'] if 'conv_dim' in config.keys() else self.hidden

        modules = []
        for i in range(self.layers):
            modules.append(ConvModule(self.conv_dim, self.walk_dim, node_feat_dim if i == 0 else self.hidden, self.hidden, self.kernel_size, config))
            if self.vn and i < self.layers - 1:
                modules.append(VNUpdate(self.hidden, config))

        self.convs = torch.nn.Sequential(*modules)

        self.node_out = torch.nn.Sequential(torch.nn.BatchNorm1d(self.hidden), torch.nn.ReLU())

        if config['graph_out'] == 'linear':
            self.linear_out = True
            self.graph_out = torch.nn.Sequential(torch.nn.Linear(self.hidden, out_dim))
        else:
            self.linear_out = False
            self.graph_out = torch.nn.Sequential(torch.nn.Linear(self.hidden, self.hidden),
                                                 torch.nn.ReLU(),
                                                 torch.nn.Linear(self.hidden, out_dim))

        pytorch_total_params = sum(p.numel() for p in self.parameters() if p.requires_grad)
        print(f'Number of paramters: {pytorch_total_params}')


    def forward(self, data, walk_steps=None, walk_start_p=1.0):
        # apply initial node feature encoding (optional)
        x_n = data.x
        if self.node_feat_enc is not None:
            x_n = self.node_feat_enc(x_n)
        data.h = x_n

        # apply initial edge feature encoding (optional)
        x_e = data.edge_attr
        if self.edge_feat_enc is not None:
            x_e = self.edge_feat_enc(x_e)

        # compute walks
        data = self.walker.sample_walks(data, x_e, steps=walk_steps, start_p=walk_start_p)

        # pre-compute array of center nodes across all walks
        data.walk_nodes_flatt = data.walk_nodes[:, self.border:-self.border].reshape(-1)

        if self.vn:
            data.vn_h = None

        # apply convolutions
        self.convs(data)

        # pool node embeddings
        data.h = self.node_out(data.h)
        if self.pool_op == 'sum':
            x = scatter_sum(data.h, data.batch, dim=0)
        else:
            x = scatter_mean(data.h, data.batch, dim=0)

        y = self.graph_out(x)
        return y

from torch_geometric.datasets import TUDataset
from torch_geometric.data import DataLoader

def get_idx(dataset, split_dir, fold):
    with open(os.path.join(split_dir, dataset, f'train_idx-{fold + 1}.txt'), 'r') as f:
        train_idx = [int(i) for i in f]
    with open(os.path.join(split_dir, dataset, f'test_idx-{fold + 1}.txt'), 'r') as f:
        test_idx = [int(i) for i in f]
    return train_idx, test_idx

from torch_geometric.utils import degree

def transform(g):
    g.x = torch.log(degree(g.edge_index[0], g.num_nodes)).view(-1, 1)
    g = preproc(g)
    return g
import math
def _split_rand(  split_ratio=0.7, seed=0, shuffle=True):
        num_entries = 1113
        indices = list(range(num_entries))
        np.random.seed(seed)
        np.random.shuffle(indices)
        split = int(math.floor(split_ratio * num_entries))
        train_idx, valid_idx = indices[:split], indices[split:]

        print(
            "train_set : test_set = %d : %d",
            len(train_idx), len(valid_idx))

        return train_idx, valid_idx
def get_split_data( config):
    dataset = TUDataset(root = '', name = 'PROTEINS', transform=transform)
    #dataset = data_name
    train_idx, test_idx = _split_rand()
    np.random.shuffle(train_idx)
    np.random.shuffle(test_idx)
    train_iter = CRaWlLoader(list(dataset[train_idx]), batch_size=50, shuffle=True)
    val_iter = CRaWlLoader(list(dataset[test_idx]), batch_size=50 , shuffle= True)

    return train_iter, val_iter

#dataset = TUDataset(name = 'PROTEINS' , root  ='')
#print(dataset[0])
#torch.manual_seed(12345)
#dataset = dataset.shuffle()
config = {
    "name": "default",
    "pool": "mean",
    "steps": 20,
    "win_size": 8,
    "walk_mode": "nb",
    "hidden_dim": 256,
    "layers": 5,
    "kernel_size": 5,
    "graph_out": "mlp",
    "dropout": 0.5,
}
train_loader , test_loader  = get_split_data(config )
#test_dataset = dataset[779:]

#train_loader = CRaWlLoader(dataset = train_dataset , shuffle  = True , batch_size  = 16)
#test_loader = CRaWlLoader(dataset = test_dataset , shuffle  = True , batch_size  = 16)



model = CRaWl(config = config, node_feat_dim = 1, edge_feat_dim = 1,out_dim =  2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

def train():
    model.train()

    for data in train_loader:  # Iterate in batches over the training dataset.
         #print(data)
         out = model(data)  # Perform a single forward pass.
         #print(out)
         #print(out)
         #print(data.y)
         #pred = out.argmax(dim=1)
         #pred = pred.view(-1)
         #print(pred.T)
         #pred = pred.unsqueeze(1)
         #print(pred)
         #y = data.y
         #y = y.squeeze(1)
         #y = y.view(-1)
         #print(y.T)
         y = data.y.squeeze(1)
         #print(y)
         loss = criterion(out, y)  # Compute the loss.
         #print('loss ',loss)
         loss.backward()  # Derive gradients.
         optimizer.step()  # Update parameters based on gradients.
         optimizer.zero_grad()  # Clear gradients.

def test(loader):
     model.eval()

     correct = 0
     total = 0
     for data in loader:  # Iterate in batches over the training/test dataset.
         out = model(data)  
         _, pred = torch.max(out.data, 1)# Use the class with highest probability.
         total += len(data.y)
         #print(pred , data.y)
         for x , y in zip(pred , data.y):
           if x == y:
             correct += 1

         #correct += (pred == data.y).sum().item() # Check against ground-truth labels.
     print(correct) 
     return  1.0*correct / total  # Derive ratio of correct predictions.


for epoch in range(50):
    train()
    #train_acc = test(train_loader)
    test_acc = test(test_loader)
    print('test Acc', {test_acc})





train_set : test_set = %d : %d 779 334
Number of paramters: 4440834
139
test Acc {0.4161676646706587}
180
test Acc {0.5389221556886228}
233
test Acc {0.6976047904191617}
232
test Acc {0.6946107784431138}
